# Webスクレイピング
## 0. 概要
Webスクレイピングとは ** データ取得 ** をする際に利用する技術  
プログラムを使ってWebからコンテンツをダウンロードすること  
例： GoogleはスクレイピングでWebページの索引を作成  

### データ取得の開発フロー
1. どのようなデータが必要かを考案  
2. そのデータを取得できるAPIがあるか調査
3. APIがなかった場合スクレイピングを実装  

#### なぜスクレイピングの優先度が低いのか
* APIの方が実装コストが低い
* Webスクレイピングは実装コストがAPIと比べて高く，法律的な制限も存在する  

### スクレイピングの工程
1. Webページにアクセス
2. ログインが必要なサイトの場合: ログイン機能の実装
3. 取得したい情報の抽出
4. 定期的に取得できるように実装

### 利用モジュール
1. webbrowser, Requests
2. Selenium
3. Beautiful Soup
4. cron  

## 1. webbrowser, Requests
### (1)webbrowser
URLを指定すると新たにブラウザを起動してWebページを開くことができる

In [1]:
import webbrowser
webbrowser.open('http://inventwithpython.com/')

True

### 利用例
* 指定した住所のGoogleMapを開きたい  
検索して開くのは面倒くさい  
* 住所をスクリプトに実行するだけでGoogleMapを開くことが可能  

In [2]:
webbrowser.open("https://www.google.com/maps/place/870+Valencia+St+San+Francisco+CA/")

True

### 応用例
* ページ上のすべてのリンクをブラウザの別々のタブで開く
* 自分の地域の天気予報を取得するURLをブラウザで開く
* 定常的にチェックする複数のSNSサイトを開く  

### (2)requests  
手軽にHTTPリクエストを送信することができるライブラリ  
ブラウザを介さずにプログラムでWebページの情報を取得することができる  
### 利用例  
* プログラムからファイルをダウンロード

In [3]:
import requests

#ダウンロードするファイルのURLを指定
url = "http://uta.pw/shodou/img/28/214.png"
savename = "test.png"

res = requests.get(url)

with open(savename, mode="wb") as f:
    f.write(res.content)
    print("保存しました")

保存しました


* Webからデータを取得してみる  

クジラ Web APIを利用して様々な情報を取得してみる  
クライアントのIPを確認するAPIを利用

In [4]:
url = "http://api.aoikujira.com/ip/ini"
res = requests.get(url)
text = res.content.decode("utf-8")
print(text)

[ip]
API_URI=http://api.aoikujira.com/ip/get.php
REMOTE_ADDR=106.185.145.165
REMOTE_HOST=M106185145165.v4.enabler.ne.jp
REMOTE_PORT=36238
HTTP_HOST=api.aoikujira.com
HTTP_USER_AGENT=python-requests/2.28.1
HTTP_ACCEPT_LANGUAGE=
HTTP_ACCEPT_CHARSET=
SERVER_PORT=80
FORMAT=ini




## 2. Beautiful Soupを利用したスクレイピング  
ページから一部のデータを取得  
サイト：https://www.nikkei.com/markets/kabu/   
このサイトから日経平均株価、始値、高値、安値の値を取得する  
タグを取得してパースしていく

### 1. URLからHTTPレスポンスを取得

In [5]:
import urllib.request
from bs4 import BeautifulSoup
#指定するURL
url = "https://www.nikkei.com/markets/kabu/"

html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, "html.parser")
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" id="R1" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="ja" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css; charset=UTF-8" http-equiv="Content-Style-Type"/>
<meta content="text/javascript; charset=UTF-8" http-equiv="Content-Script-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="nodescription, noimage" name="mixi-check-robots"/>
<meta content="" name="thumbnail">
<meta content="" property="og:image"/>
<meta content="197388106979545" property="fb:app_id"/>
<meta content="日経電子版の総合投資・金融情報コーナー。株式・為替から債券、商品、新興国市場など国内外の最新マーケット情報はもちろん、第一線の記者による分析・解説記事を豊富に提供します。企業の最新ニュースや詳細な株価・財務データを提供するオンライン版の「日経会社情報」、チャートや銘柄管理ツールなどの便利な機能も充実しています。" name="description"/>
<meta content="日経,日本経済新聞,日経電子版,nikkei,日経平均,株価,株式,銘柄,金融,投資,マネー,マ

### 2. ソースから取得したい箇所を抽出していく

#### findメソッドでタグとクラスの種類を指定  
該当した最初の部分を取得することができる  

In [6]:
market = soup.find('div', class_='mk-top_stock_average cmn-clearfix')
market

<div class="mk-top_stock_average cmn-clearfix">
<div class="m-headline">
<h2 class="m-headline_text">日経平均株価<span class="cmn-notes">（7日、大引、円）</span></h2>
</div>
<div class="cmn-clearfix">
<div class="mkc-chart">
<a href="/smartchart/" style="text-decoration:none;" target="_blank"><img class="lazyload" data-original="https://chart-renderer.n8s.jp/m/marketdata/indexes/line/tick/?code=N101/T&amp;chart_style=pc_half&amp;term=1&amp;Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jaGFydC1yZW5kZXJlci5uOHMuanAvbS9tYXJrZXRkYXRhKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY2NTM4NjUyMH19fV19&amp;Signature=U8SrBRc2s4h7yXQjkGw5vD2cVM2rhgOc1RsWkIAFiZV1Oy6373rDCCI77MpVjqRNSZLqKloPvktV-NuS34hqWAMkYnqbqNo6AiJt6M1OGUtNK-hyA4NK0o1pWCtZ89JE1PENvPC~rISjP4SWShY-amxVlvNhOmtYRWuSgAhpgVKnraC3fFWS~7Dsg0Efe8zDyf~JPrA6EybExmlNaiounayTAGmFh0s4JY91bRCK6dEQ9pN~imKgXwkGKl1G-DWjlhwTZHQYyhXwvxN5TcKtl3fo4KXFJAHYJz3X~kOHD-sKTN7aHG4xbHQIzoyyUDbOyPl5oRYWJ7aQw4jdlIinGA__&amp;Key-Pair-Id=APKAJWFGMKS

#### find_allメソッドで該当する全ての部分をリストにして取得  

In [7]:
prices = market.find_all('dd')
print(prices)
print(prices[0].text)

[<dd>26,975.92</dd>, <dd>27,198.91</dd>, <dd>26,921.90</dd>]
26,975.92


## 演習 

### その1
以下のサイトの国内株式の種類とその値をリストにしましょう  
https://www.nikkei.com/markets/worldidx/  

### その2
テーブルを辞書型にして保存してみよう。  
dictionary {"日経平均株価": 23710.22, ....}  